In [ ]:
import cards
from typing import List

cards.LOGGING_ENABLED = True

def print_tree(state:cards.Player, depth = 0):
    print ("  "*depth, state.short_str())
    for child in state.childstates:
        print_tree(child, depth+1)


def get_all_leaf_nodes(state:cards.Player) -> List[cards.Player]:
    if state.is_pruned:
        return []

    if len(state.childstates) == 0:
        return [state]
    else:
        leaf_nodes = []
        for child in state.childstates:
            leaf_nodes.extend(get_all_leaf_nodes(child))
        return leaf_nodes


In [ ]:
decklist = """
1 Arboreal Grazer
1 Llanowar Elves
1 Elvish Mystic
1 Sol Ring
1 Arbor Elf
2 Sakura-Tribe Elder
4 Wall of Roots
4 Chancellor of the Tangle
1 Panglacial Wurm
1 Abundant Harvest
4 Ancient Stirrings
4 Caravan Vigil
2 Lay of the Land
4 Reclaim the Wastes
3 Explore
4 Land Grant
3 Rampant Growth
2 Nissa's Pilgrimage
4 Recross the Paths
2 Search for Tomorrow
4 Goblin Charbelcher
4 Wild Growth
7 Forest
"""

In [ ]:

# Create a player with a predictable seed
player = cards.Player(decklist, 12)
player.start_game()

player.start_turn()
# Ensure that the player has 7 cards in hand
assert len(player.hand) == 7, f"Player should have 7 cards in hand, has {len(player.hand)}"

# Get a Search for Tomorrow
search = player.debug_force_get_card_in_hand("Search for Tomorrow")

# Cast the card for its alternate cost
player.mana_pool += search.alt_cost # Cheat and add mana to pay costs
player.alt_play(search)

# Pass the turn
player.start_turn()
# Ensure that we still have no lands
assert player.lands == 0
assert player.mana_pool == 0

# Pass the turn again
player.start_turn()
# Ensure that we now have 1 land, and it is untapped
assert player.lands == 1
assert player.mana_pool == 1

# Now test casting Search for Tomorrow for its full cost.
search = player.debug_force_get_card_in_hand("Search for Tomorrow")
print(f'player.has_mana: {player.has_mana(search.cost, search.colorless_cost)}')
player.mana_pool += search.cost
player.play(search)
# Ensure that we now have 2 lands, both untapped
assert player.lands == 2
assert player.mana_pool == 2


In [ ]:

# Create a player with a predictable seed
player = cards.Player(decklist, 12)
player.start_game()
player.start_turn()
# Ensure that the player has 7 cards in hand
assert len(player.hand) == 7

# Get a Sol Ring
solring = player.debug_force_get_card_in_hand("Sol Ring")
belcher = player.debug_force_get_card_in_hand("Goblin Charbelcher")
search = player.debug_force_get_card_in_hand("Search for Tomorrow")

# Cast the Sol Ring
player.mana_pool += 2 # Cheat and add mana to pay costs
player.play(solring)

assert player.colorless_lands == 2
assert player.colorless_mana_pool == 2

# Cast the Search for Tomorrow
player.play(search)

# Ensure that we now have 1 land, and it is untapped.
assert player.mana_pool == 1
assert player.colorless_mana_pool == 0

# Pass the turn
player.start_turn()

assert player.lands == 1
assert player.mana_pool == 1
assert player.colorless_mana_pool == 2

# Cheat and add some mana
player.mana_pool += 4
player.play(belcher)

# Ensure that we spent our colorless mana before our colored mana
assert player.mana_pool == 3
assert player.colorless_mana_pool == 0


In [ ]:

# Create a player with a predictable seed
player = cards.Player(decklist, 14)
player.start_turn()
# Ensure that the player has 7 cards in hand
assert len(player.hand) == 7

# Get a Land Grant
card = player.debug_force_get_card_in_hand("Land Grant")

print(card.long_str(player))

# Cast the card for its alternate cost
assert player.hand.count_cards("Forest") == 0

assert not card.can_play(player)
assert card.can_alt_play(player)

player.alt_play(card)

assert player.hand.count_cards("Forest") == 1


In [ ]:
# Test the AI's ability to cast Land Grant when needed

# Create a player with a predictable seed
player = cards.Player(decklist, 14)
player.start_turn()
# Ensure that the player has 7 cards in hand
assert len(player.hand) == 7

# Get a Search for Tomorrow
card = player.debug_force_get_card_in_hand("Land Grant")

print(f'Starting state:')
print(player)
print()

player.step_next_actions()

for i in range(5):
    leafs = get_all_leaf_nodes(player)
    for leaf in leafs:
        if leaf.current_turn == 1:
            leaf.step_next_actions()

print_tree(player)

leafs = get_all_leaf_nodes(player)
for i, leaf in enumerate(leafs):
    print()
    print('********')
    print(f'Leaf #{i}: {leaf.short_str()}')
    print()
    leaf.dumplog()
    print()
    print(leaf)

In [ ]:
player.dumplog()
print()
print(player)